In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [49]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [50]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.467625
Minibatch accuracy: 12.5%
Validation accuracy: 12.4%
Minibatch loss at step 100: 11.781839
Minibatch accuracy: 73.4%
Validation accuracy: 73.0%
Minibatch loss at step 200: 4.831479
Minibatch accuracy: 75.8%
Validation accuracy: 76.6%
Minibatch loss at step 300: 2.088650
Minibatch accuracy: 82.0%
Validation accuracy: 80.2%
Minibatch loss at step 400: 1.082435
Minibatch accuracy: 82.8%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.978718
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 600: 1.087825
Minibatch accuracy: 74.2%
Validation accuracy: 80.6%
Minibatch loss at step 700: 1.129576
Minibatch accuracy: 71.1%
Validation accuracy: 81.6%
Minibatch loss at step 800: 0.727669
Minibatch accuracy: 84.4%
Validation accuracy: 78.1%
Minibatch loss at step 900: 0.749297
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 1000: 0.877003
Minibatch accuracy: 80.5%
Validation accuracy: 80.

In [24]:
# neural network with 1 hidden layer
batch_size = 128
hidden_layer = 1024
beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer]))
  biases = tf.Variable(tf.zeros([hidden_layer]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  logits1 = tf.matmul(tf_train_dataset, weights) + biases
  relus = tf.nn.relu(logits1)
  # Training computation.
  logits2 = tf.matmul(relus, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) + beta*tf.nn.l2_loss(weights2)
  
#   # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 297.283997
Minibatch accuracy: 13.3%
Validation accuracy: 34.9%
Minibatch loss at step 500: 13.579014
Minibatch accuracy: 80.5%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 9.070143
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 7.339707
Minibatch accuracy: 82.0%
Validation accuracy: 78.4%
Minibatch loss at step 2000: 3.733864
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 5.442884
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 6.095272
Minibatch accuracy: 80.5%
Validation accuracy: 79.2%
Test accuracy: 85.9%


In [26]:
# Problem 2 - Overfitting

num_steps = 3001
num_diff_batches = 10

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (batch_size *10)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, prediction = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 328.787842
Minibatch accuracy: 12.5%
Validation accuracy: 42.7%
Minibatch loss at step 500: 2.556619
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1000: 1.546507
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.936198
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.567104
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2500: 0.343793
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Test accuracy: 83.6%


In [31]:
# Introducing dropout
batch_size = 128
hidden_layer = 1024
dropout_prob = 0.5
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Input -> Hidden Layer
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer]))
  hidden_biases = tf.Variable(tf.zeros([hidden_layer]))
  hidden_weights_after_dropout = tf.nn.dropout(hidden_weights, dropout_prob)
  hidden_logits = tf.matmul(tf_train_dataset, hidden_weights_after_dropout) + hidden_biases
  relus = tf.nn.relu(hidden_logits)

  # Hidden Layer -> Output
  weights = tf.Variable(
    tf.truncated_normal([hidden_layer, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  weights_after_dropout = tf.nn.dropout(weights, dropout_prob)
  logits = tf.matmul(relus, weights_after_dropout) + biases


  # Training computation.

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
#   # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (batch_size * 10)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 721.862610
Minibatch accuracy: 6.2%
Validation accuracy: 23.6%
Minibatch loss at step 500: 6.151092
Minibatch accuracy: 97.7%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 9.199118
Minibatch accuracy: 96.1%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 2.196368
Minibatch accuracy: 97.7%
Validation accuracy: 80.1%
Minibatch loss at step 2000: 5.956400
Minibatch accuracy: 98.4%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 3.318865
Minibatch accuracy: 97.7%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Test accuracy: 87.2%


In [106]:
# Deep network
batch_size = 1024
layers = [1024, 1024, num_labels]
dropout_prob = 0.5
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size), name="train_dataset")
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="train_labels")
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # hidden layers
  weights = []
  biases = []
  logits = []
  relus = []
  n = image_size*image_size
  for i in range(len(layers)):
      m = layers[i]
      name_w, name_l = "Weights" + str(i), "Logits" + str(i)
      weights.append(tf.Variable(tf.truncated_normal([n, m]), name=name_w))
      biases.append(tf.Variable(tf.zeros([m])))
      if (i==0):
        logits.append(tf.matmul(tf_train_dataset, weights[0], name=name_l) + biases[0])
      else:
        logits.append(tf.matmul(relus[i-1], weights[i], name=name_l) + biases[i])
      relus.append(tf.nn.relu(logits[i]))
      n = layers[i]

#   hidden_weights = tf.Variable(
#     tf.truncated_normal([image_size * image_size, hidden_layer]))
#   hidden_biases = tf.Variable(tf.zeros([hidden_layer]))
#   hidden_weights_after_dropout = tf.nn.dropout(hidden_weights, dropout_prob)
#   hidden_logits = tf.matmul(tf_train_dataset, hidden_weights_after_dropout) + hidden_biases
#   relus = tf.nn.relu(hidden_logits)

#   # Last Hidden Layer -> Output
#   weights = tf.Variable(
#     tf.truncated_normal([hidden_layer, num_labels]))
#   biases = tf.Variable(tf.zeros([num_labels]))
#   weights_after_dropout = tf.nn.dropout(weights, dropout_prob)
#   logits = tf.matmul(relus, weights_after_dropout) + biases


  # Training computation.

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits[-1], tf_train_labels), name="loss")
  global_step = tf.Variable(0)
  starter_learning_rate = 0.01
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.90, staircase=True)
#   # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
  
#   # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits[-1])

#   valid_prediction = tf.nn.softmax(
#     tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
#   test_prediction = tf.nn.softmax(
#     tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [107]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  train_writer = tf.train.SummaryWriter('.',
                                      session.graph)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % (num_steps/10) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#       print("Validation accuracy: %.1f%%" % accuracy(
#         valid_prediction.eval(), valid_labels))
#   print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7844.619141
Minibatch accuracy: 9.7%
Minibatch loss at step 300: 167.178864
Minibatch accuracy: 81.7%
Minibatch loss at step 600: 83.500320
Minibatch accuracy: 84.6%
Minibatch loss at step 900: 70.737381
Minibatch accuracy: 84.6%
Minibatch loss at step 1200: 43.078529
Minibatch accuracy: 86.3%
Minibatch loss at step 1500: 37.826939
Minibatch accuracy: 85.4%
Minibatch loss at step 1800: 23.932320
Minibatch accuracy: 88.4%
Minibatch loss at step 2100: 37.806168
Minibatch accuracy: 84.7%
Minibatch loss at step 2400: 20.553539
Minibatch accuracy: 86.2%
Minibatch loss at step 2700: 25.627275
Minibatch accuracy: 84.7%
Minibatch loss at step 3000: 19.762720
Minibatch accuracy: 85.9%


In [105]:
logits

[<tf.Tensor 'add:0' shape=(128, 1024) dtype=float32>,
 <tf.Tensor 'add_1:0' shape=(128, 1024) dtype=float32>,
 <tf.Tensor 'add_2:0' shape=(128, 10) dtype=float32>]